# <a id='toc1_'></a>[SPEARHEAD VDA tool](#toc0_)

The tool receives a list of time references and outputs the corresponding plots derived from the VDA analysis based on user input.

*The notebook cells are meant to be run serially. In cases where this is not applicable, the user is warned and instructed beforehand via the markdown cells.*

## <a id='toc1_1_'></a>[Imports & Setup](#toc0_)

The "Imports & Setup" section imports dependencies used throughout the tool and also indirectly defines constants and default values.

In [ ]:
from warnings import simplefilter, filterwarnings
from pandas.errors import PerformanceWarning
from astropy.visualization import quantity_support

from vda_tool_configuration import VDA_parameters
from vda_views import VDA_nb_displayer
from vda import VDA

# omit Pandas' PerformanceWarning
simplefilter(action='ignore', category=PerformanceWarning)
filterwarnings(action='ignore', message="Discarding nonzero nanoseconds in conversion")

The SPEARHEAD VDA tool relies on three classes: (a) VDA, (b) VDA_parameters, and (c) VDA_nb_displayer.

The first one holds the core functions of the VDA tool, the second one stores the parameters under which the VDA calculations are made, while the third contains functions that help with the notebook vidualizations

In [ ]:
vda_parameters = VDA_parameters()
vda = VDA(vda_parameters)
vda_displayer = VDA_nb_displayer(vda)

## <a id='toc1_2_'></a>[Parameterize](#toc0_)

This section parameterizes the notebook. The user should run the code cells one at a time, change the variables as they desire, and proceed to the next when the input fields are properly filled.

In each cell there is a brief description for the asked inputs.

After each form group the adjusted variables are displayed when the immediately next cell(s) run(s).

*Note: In case prior input needs to be changed, all the input cells after the changed one shall be rerun to avoid inconsistencies.*

### <a id='toc1_2_6_'></a>[Views/Displays](#toc0_)

- Display the produced DataFrames: this should be checked to display the affected variables/DataFrames

In [ ]:
vda_displayer.display_view_toggle()

### <a id='toc1_2_1_'></a>[Event time(s)](#toc0_)

There are three options for the gathering of the events' data.
- Custom datetime range: provide a starting and an ending date for the data collection. An event should start between these dates for the calculations tobe meaningful
- File with datetime ranges: same concept as the above, but for multiple events. Each line should have a starting and an ending datetime (see `examples/datetime_range_example.csv` file)
- File with reference datetimes: A file containing a list of datetimes (see `examples/reference_times_example.csv` file). Two extra parameters are defined to define a datetime range (see next Markdown cell)

In [ ]:
vda_displayer.display_input_type()

(Applicable only if *File with reference datetimes* is selected)

The reference datetimes and the rest of the parameters of this section define the time window for which to download particle data.

- Reference times file: the path to the file which states the reference datetimes. This must be a .csv file with 2 columns. The first column is an `id` for the reference (event), while the second column is the reference datetime itself. The datetime format should be any format that is supported by the `pandas.to_pydatetime` function (i.e. 2024-12-31 00:00:00). The first line of the .csv file should be the headers, although the names of the headers don't matter to the rest of the tool.

- Hours prior to the reference time: an integer indicating hours **before** the reference datetime. The derived datetime is the **start** of the time window for the particle data download.

- Hours after the reference time: an integer indicating hours **after** the reference datetime. The derived datetime is the **end** of the time window for the particle data download.

In [ ]:
vda_displayer.display_date_range()

In [ ]:
vda.construct_times_df()

### <a id='toc1_2_2_'></a>[Particle Data](#toc0_)

There is an option to load arleady downloaded data. In case the user has a proper .pkl file (outputted by this notebook) and wants to load the data from it, the "Load data" checkbox should be checked.

- In case of "Load data" the user should provide the path to the .pkl file

- In case of a run with new data: the user should state if he wants the data to be saved by checking the "Save data" checkbox and providing a path for the outputted file. The path should state a .pkl file to be created.

In [ ]:
vda_displayer.display_load_data_option()

In [ ]:
vda_displayer.display_save_data_option()

The user should check the sensor/particle species combinations and viewings of the data to be downloaded.
- Resample frequency: the frequency for the data to be resampled. This should be provided as an **offset alias**. [Reference](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases) for instructions and examples.

**Important Note: In case of data load the checked items and frequency should match the loaded dataset**

In [ ]:
vda_displayer.display_particle_selection()

In [ ]:
vda.construct_particles_df()

### <a id='toc1_2_3_'></a>[Energy Channels Grouping](#toc0_)

The respective energies for the downloaded data:

In [ ]:
vda.construct_energies_df()

The energy grouping method is the following:

For each series, multiply the values with the respective bin width and sum it all together. Then divide the result with the total bin width (sum of bin widths or max of last energy channel - min of first energy channel).

- `<species>`: the number of `<species>` energy channels to be combined into a single energy channel.

In [ ]:
vda_displayer.display_groupings()

In [ ]:
vda.group_energy_channels()

### <a id='toc1_2_4_'></a>[Onset Determination Method](#toc0_)

- Onset determination method: the method to be utilized for the onset determination.

In [ ]:
vda_displayer.display_onset_method_selection()

The method-specific parameters for the onset determination. For details refer to the relative method.

In [ ]:
vda_displayer.display_onset_method_parameters()

In [ ]:
vda.plot_bg_selection()

In [ ]:
vda.calculate_onsets()

In [ ]:
vda.clean_onsets()

### <a id='toc1_2_5_'></a>[Onset Selection](#toc0_)

In [ ]:
vda.construct_options_df()
# internal dataframe used for the onset selection

- Onset determination method: choose how the onset times that will be used for the VDA plots should be chosen. Options:
  - Use all: every determined onset time will be used
  - Interactive: the user is prompted to decide whether to use the determined onset (by viewing) or not individually per grouped energy channel
  - Custom list: The user should construct and provide a list defining which viewing should be used per energy channel. The list items should follow the same order as the one of the displayed plots. *(under development)*

In [ ]:
vda_displayer.display_onset_selection_selection()

In [ ]:
vda_displayer.select_onsets()

In [ ]:
vda.parameters.selected_onsets

## <a id='toc1_3_'></a>[VDA](#toc0_)

### <a id='toc1_3_5_1_'></a>[Channels](#toc0_)

In [ ]:
vda.construct_energy_channels_characteristics()

### <a id='toc1_3_5_2_'></a>[Spacecraft](#toc0_)

In [ ]:
quantity_support()

In [ ]:
vda.define_spacecraft_parameters()

### <a id='toc1_3_5_3_'></a>[Plots](#toc0_)

In [ ]:
vda.plot()